<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction-to-Unix---awk-and-Makefiles" data-toc-modified-id="Introduction-to-Unix---awk-and-Makefiles-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction to Unix - awk and Makefiles</a></span></li><li><span><a href="#Working-with-tabular-files:-Awk" data-toc-modified-id="Working-with-tabular-files:-Awk-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Working with tabular files: Awk</a></span><ul class="toc-item"><li><span><a href="#Example-of-tabular-file:-the-GFF3-format" data-toc-modified-id="Example-of-tabular-file:-the-GFF3-format-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Example of tabular file: the GFF3 format</a></span></li><li><span><a href="#Basic-AWK-syntax:-filters" data-toc-modified-id="Basic-AWK-syntax:-filters-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Basic AWK syntax: filters</a></span><ul class="toc-item"><li><span><a href="#Exercise" data-toc-modified-id="Exercise-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Exercise</a></span></li></ul></li><li><span><a href="#Awk:-printing-columns-and-doing-operations" data-toc-modified-id="Awk:-printing-columns-and-doing-operations-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Awk: printing columns and doing operations</a></span><ul class="toc-item"><li><span><a href="#Exercise-(difficult)" data-toc-modified-id="Exercise-(difficult)-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Exercise (difficult)</a></span></li></ul></li><li><span><a href="#AWK:-searching-by-regular-expressions" data-toc-modified-id="AWK:-searching-by-regular-expressions-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>AWK: searching by regular expressions</a></span><ul class="toc-item"><li><span><a href="#Last-exercise!" data-toc-modified-id="Last-exercise!-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Last exercise!</a></span></li></ul></li></ul></li><li><span><a href="#Bonus:-Makefiles" data-toc-modified-id="Bonus:-Makefiles-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bonus: Makefiles</a></span><ul class="toc-item"><li><span><a href="#Defining-pipelines-with-Makefiles" data-toc-modified-id="Defining-pipelines-with-Makefiles-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Defining pipelines with Makefiles</a></span></li><li><span><a href="#How-to-run-Makefile-rules" data-toc-modified-id="How-to-run-Makefile-rules-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>How to run Makefile rules</a></span></li><li><span><a href="#Dinner-time!!" data-toc-modified-id="Dinner-time!!-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Dinner time!!</a></span></li></ul></li></ul></div>

In [67]:
# Configuration - please ignore - this should not appear in the slideshow
alias grep='grep --color'
cd
cd workspace/Peb2019/exercises/

# Introduction to Unix - awk and Makefiles

**<font color='darkgreen'> Welcome to the Programming for Evolutionary Biology workshop!! </font>** 

Giovanni M. Dall'Olio and Alvaro Perdomo-Sabogal, 03/03/2019. Quick link to slides online: http://tinyurl.com/evop2019


All materials available here: https://github.com/dalloliogm/evop2019/archive/master.zip


In this fourth part we will use the **awk** command to excplore the contents of files, and some basic regular expressions.

Press space or down key to continue.

# Working with tabular files: Awk


The **awk** command allows to search and manipulate tabular files from the command line.

Imagine it as the equivalent of Excel/Calc for the command line. It allows to do search on specific columns of a file, to do numerical operations, or to change the order of the columns.

The advantage of a command-line tool over graphical software is that the memory footprint is much lower. So you can access and modify large files in a fraction of the time that it would take with Excel.

## Example of tabular file: the GFF3 format

The file genes/chr8.gff contains an example of file in the GFF3 format:

In [68]:
head genes/chr8.gff

##gff-version 3
##source-version rtracklayer 1.34.1
##date 2017-02-27
##genome-build .	hg19
chr8	rtracklayer	sequence_feature	18248755	18258723	.	+	.	gene_id=10;symbol=NAT2;exerc1=FALSE;ID=10
chr8	rtracklayer	sequence_feature	100549014	100549089	.	-	.	gene_id=100126309;symbol=MIR875;exerc1=FALSE;ID=100126309
chr8	rtracklayer	sequence_feature	144895127	144895212	.	-	.	gene_id=100126338;symbol=MIR937;exerc1=FALSE;ID=100126338
chr8	rtracklayer	sequence_feature	145619364	145619445	.	-	.	gene_id=100126351;symbol=MIR939;exerc1=FALSE;ID=100126351
chr8	rtracklayer	sequence_feature	91970706	91997485	.	-	.	gene_id=100127983;symbol=C8orf88;exerc1=FALSE;ID=100127983
chr8	rtracklayer	sequence_feature	74332309	74353753	.	+	.	gene_id=100128126;symbol=STAU2-AS1;exerc1=FALSE;ID=100128126


As you can see it is a tab-separated file, which we could easily read in Excel or Calc.

The **GFF** (General Feature Format) format specifications are defined [here](https://genome.ucsc.edu/FAQ/FAQformat.html#format3), but in short:

- the **col1**, **col4** and **col5** contain the chromosome name and genomic coordinates (start and end),
- the **col2** describes the tool or resource that generated the annotation,
- the **col3** describe the type of feature (e.g. gene, transcript, exon, TF binding site, Histone Acetylation mark, etc...
- the **col9** column contains several fields, separated by a semicolon


## Basic AWK syntax: filters

The basic AWK syntax is the following:


    awk 'PATTERN {ACTION}' filename

Awk is quite smart at recognizing the field separator, and by default assumes they are separated by tabs.

    - PATTERN: select which rows to print
        e.g. print only the rows that contain a specific pattern
        
    - ACTION: what to do on each line (e.g. print a specific column, sum them, etc..)
        Columns are identified by $column_number, e.g. $2 refers to the second column, and so on. 
        $0 refers to all columns

The following code filters all the lines belonging to chromosome 8, between the coordinates 100000 and 200000:

In [72]:
# This prints all the lines on chromosome 8 and with chromosomal position between 10000 and 20000
# Don't be scared by this long statement, we will explain it step by step
# 
awk '$1=="chr8" && $4>100000 && $5<200000 {print $0}' genes/chr8.gff

chr8	rtracklayer	sequence_feature	182200	197339	.	+	.	gene_id=169270;symbol=ZNF596;exerc1=FALSE;ID=169270
chr8	rtracklayer	sequence_feature	116086	117024	.	-	.	gene_id=441308;symbol=OR4F21;exerc1=FALSE;ID=441308
chr8	rtracklayer	sequence_feature	158345	182318	.	-	.	gene_id=644128;symbol=RPL23AP53;exerc1=FALSE;ID=644128


### Exercise

Can you print all the lines between chromosomal positions 5,000,000 and 10,000,000 (columns 4 and 5)?

In [70]:
awk '$4 > 5000000 && $5 < 10000000 ' genes/chr8.gff | head


chr8	rtracklayer	sequence_feature	7143733	7212876	.	-	.	gene_id=100128890;symbol=FAM66B;exerc1=TRUE;ID=100128890
chr8	rtracklayer	sequence_feature	7215498	7220490	.	-	.	gene_id=100131980;symbol=ZNF705G;exerc1=TRUE;ID=100131980
chr8	rtracklayer	sequence_feature	7812535	7866277	.	+	.	gene_id=100132103;symbol=FAM66E;exerc1=TRUE;ID=100132103
chr8	rtracklayer	sequence_feature	7783859	7809935	.	+       / Cows in \
chr8	rtracklayer	sequence_feature	6261077	6264069	.	        | the      |
chr8	rtracklayer	sequence_feature	7272385	7274354	.	-       \ Genome!  /
chr8	rtracklayer	sequence_feature	7946463	7946611	.	.	  ---------
chr8	rtracklayer	sequence_feature	6602685	6602765	.	+	 ||  ^__^
chr8	rtracklayer	sequence_feature	8905955	8906028	.	+	 ||  (oo)\\_______
chr8	rtracklayer	sequence_feature	6602689	6602761	.	-	     (__)\\       )\\/\


## How many genes there are?

In the previous exercises we've found all the genes on chr8 and between position 5000,000 and 10,000,000.

How many genes are these?


With grep, we've used the -c option to count the output lines. However, awk doesn't have a -c option. 

How to count the output lines, then?

A solution is to use the **wc** unix command, which stands for "word-count":

In [71]:
awk '$4 > 5000000 && $5 < 10000000 ' genes/chr8.gff | wc 


      47     425    4709


The first number is the number of rows; the other numbers are the number of words, and the number of characters in total.


## Awk: printing columns and doing operations

Awk also allows to print only specific columns, and do algebraic operations on them.

Remember that each column can be referred as \\$1, \\$2, \\$3, etc...

For example the following code prints the chromosome, start and end position for each gene (columns 1, 4 and 5), as well as column 9 which contains the gene symbol and ids. We can pipe the output to head or less, to make it easier to visualize:

In [62]:
awk '{print $1, $4, $5, $9}' genes/chr8.gff | head


##gff-version   
##source-version   
##date   
##genome-build   
chr8 18248755 18258723 gene_id=10;symbol=NAT2;exerc1=FALSE;ID=10
chr8 100549014 100549089 gene_id=100126309;symbol=MIR875;exerc1=FALSE;ID=100126309
chr8 144895127 144895212 gene_id=100126338;symbol=MIR937;exerc1=FALSE;ID=100126338
chr8 145619364 145619445 gene_id=100126351;symbol=MIR939;exerc1=FALSE;ID=100126351
chr8 91970706 91997485 gene_id=100127983;symbol=C8orf88;exerc1=FALSE;ID=100127983
chr8 74332309 74353753 gene_id=100128126;symbol=STAU2-AS1;exerc1=FALSE;ID=100128126


Notice how this also prints the headers of the file. We can exclude these by adding a grep condition:

In [64]:
# Let's use grep and the -v option to exclude all the lines containing #
grep -v '#' genes/chr8.gff |  awk '{print $1, $4, $5, $9}'  |  head

chr8 18248755 18258723 gene_id=10;symbol=NAT2;exerc1=FALSE;ID=10
chr8 100549014 100549089 gene_id=100126309;symbol=MIR875;exerc1=FALSE;ID=100126309
chr8 144895127 144895212 gene_id=100126338;symbol=MIR937;exerc1=FALSE;ID=100126338
chr8 145619364 145619445 gene_id=100126351;symbol=MIR939;exerc1=FALSE;ID=100126351
chr8 91970706 91997485 gene_id=100127983;symbol=C8orf88;exerc1=FALSE;ID=100127983
chr8 74332309 74353753 gene_id=100128126;symbol=STAU2-AS1;exerc1=FALSE;ID=100128126
chr8 144816310 144828507 gene_id=100128338;symbol=FAM83H-AS1;exerc1=FALSE;ID=100128338
chr8 144077245 144079080 gene_id=100128627;symbol=CDC42P3;exerc1=FALSE;ID=100128627
chr8 30239635 30242917 gene_id=100128750;symbol=RBPMS-AS1;exerc1=FALSE;ID=100128750
chr8 7143733 7212876 gene_id=100128890;symbol=FAM66B;exerc1=TRUE;ID=100128890


### Doing operations on awk columns (adding gene length)

We can also do operations on the columns printed by awk.

For example, we can calculate the gene length by subtracting the gene start position from the gene end

In [65]:
# Calculating gene length
grep -v '#' genes/chr8.gff |  awk '{print $1, $4, $5, $5-$4, $9}'  |  head

chr8 18248755 18258723 9968 gene_id=10;symbol=NAT2;exerc1=FALSE;ID=10
chr8 100549014 100549089 75 gene_id=100126309;symbol=MIR875;exerc1=FALSE;ID=100126309
chr8 144895127 144895212 85 gene_id=100126338;symbol=MIR937;exerc1=FALSE;ID=100126338
chr8 145619364 145619445 81 gene_id=100126351;symbol=MIR939;exerc1=FALSE;ID=100126351
chr8 91970706 91997485 26779 gene_id=100127983;symbol=C8orf88;exerc1=FALSE;ID=100127983
chr8 74332309 74353753 21444 gene_id=100128126;symbol=STAU2-AS1;exerc1=FALSE;ID=100128126
chr8 144816310 144828507 12197 gene_id=100128338;symbol=FAM83H-AS1;exerc1=FALSE;ID=100128338
chr8 144077245 144079080 1835 gene_id=100128627;symbol=CDC42P3;exerc1=FALSE;ID=100128627
chr8 30239635 30242917 3282 gene_id=100128750;symbol=RBPMS-AS1;exerc1=FALSE;ID=100128750
chr8 7143733 7212876 69143 gene_id=100128890;symbol=FAM66B;exerc1=TRUE;ID=100128890


## AWK: searching by regular expressions

Awk can also be used to search by regular expression.

For example, the following code will print all the lines in which coumn \\$9 contains "MIR":

In [73]:
awk '$9 ~ /MIR/ {print $0}' genes/chr8.gff  | head

chr8	rtracklayer	sequence_feature	100549014	100549089	.	-	.	gene_id=100126309;symbol=MIR875;exerc1=FALSE;ID=100126309
chr8	rtracklayer	sequence_feature	144895127	144895212	.	-	.	gene_id=100126338;symbol=MIR937;exerc1=FALSE;ID=100126338
chr8	rtracklayer	sequence_feature	145619364	145619445	.	-	.	gene_id=100126351;symbol=MIR939;exerc1=FALSE;ID=100126351
chr8	rtracklayer	sequence_feature	65285775	65295842	.	+	.	gene_id=100130155;symbol=MIR124-2HG;exerc1=FALSE;ID=100130155
chr8	rtracklayer	sequence_feature	128972879	128972941	.	+	.	gene_id=100302161;symbol=MIR1205;exerc1=FALSE;ID=100302161
chr8	rtracklayer	sequence_feature	10682883	10682953	.	-	.	gene_id=100302166;symbol=MIR1322;exerc1=FALSE;ID=100302166
chr8	rtracklayer	sequence_feature	129021144	129021202	.	+	.	gene_id=100302170;symbol=MIR1206;exerc1=FALSE;ID=100302170
chr8	rtracklayer	sequence_feature	129061398	129061484	.	+	.	gene_id=100302175;symbol=MIR1207;exerc1=FALSE;ID=100302175
chr8	rtracklayer	sequence_feature	128808208	12880827

### Exercise!

Calculate the lenght of the gene POU5F1B.

Find the Gene whose gene_id is equal to that number.

In [13]:
awk '$9 ~ /gene_id=1584/ {print $0}' genes/chr8.gff 

chr8	Well_done	Great_job	143953773	143961236	.	-	.	gene_id=1584;symbol=CYP11B1;exerc1=FALSE;ID=1584


In [12]:
awk '$9 ~ /POU5F1B/ {print $5-$4}' genes/chr8.gff 


1584


## Splitting a file in multiple files, according to value of a column

The example.bed file contains lines belonging to different chromosomes (column 1). 

We can split these in separate files, using the following syntax:

In [75]:
#awk '{print>$1".txt"}' example.bed # doesn't work on MacBooc
awk '{print>$1}' genes/example.bed

In [76]:
ls

1_browsing_textfiles.txt	chr8
2_searching_patterns.txt	genes
bos_taurus.txt			multiplefiles
chr20				old_files


### Extracting gene symbols from a string in awk

Notice how in in this file the gene symbols are embedded in a long string containing other information:

    gene_id=10;symbol=NAT2;exerc1=FALSE;ID=10


Is there a way to extract just the gene ID and Symbols from this text, while also keeping the gene coordinates?


Extracting the gene symbols in this case is not trivial, but this example applies to several formats in bioinformatics, so it is useful to see how to do it.

The first step is to use the -F operator in awk to specify a different fields separator - in this case, the semicolon:



In [54]:
grep -v '^#' genes/chr8.gff | awk -F';' '{print $2}' | head

symbol=NAT2
symbol=MIR875
symbol=MIR937
symbol=MIR939
symbol=C8orf88
symbol=STAU2-AS1
symbol=FAM83H-AS1
symbol=CDC42P3
symbol=RBPMS-AS1
symbol=FAM66B


### How to print both gene coordinates and gene symbols?

We have to use two consecutive awk searches. 

- in the first awk statement, we will extract the chromosome, start, end columns, as well as the string containing the symbol string (column 1, 4,5, and 9);
- in the second statement, we will split the symbol string to extract the symbol id  

In [58]:
grep -v '^#' genes/chr8.gff | awk '{print $1, $4, $5, $9}' | grep -v '^#' | awk -F';' '{print $1, $2}' | head

chr8 18248755 18258723 gene_id=10 symbol=NAT2
chr8 100549014 100549089 gene_id=100126309 symbol=MIR875
chr8 144895127 144895212 gene_id=100126338 symbol=MIR937
chr8 145619364 145619445 gene_id=100126351 symbol=MIR939
chr8 91970706 91997485 gene_id=100127983 symbol=C8orf88
chr8 74332309 74353753 gene_id=100128126 symbol=STAU2-AS1
chr8 144816310 144828507 gene_id=100128338 symbol=FAM83H-AS1
chr8 144077245 144079080 gene_id=100128627 symbol=CDC42P3
chr8 30239635 30242917 gene_id=100128750 symbol=RBPMS-AS1
chr8 7143733 7212876 gene_id=100128890 symbol=FAM66B


# Bonus: Makefiles


You may have noticed that the unix_intro folder contains a file called Makefile. 
 
What is a Makefile? Let's have a look at its contents first: 

In [14]:
cd ..
head Makefile

# This is a Makefile, which will be explained later in the course.
# Please don't look at it yet :-)

publish: slides_bash commit
	echo "convert the slides to pdf, commit, and push to github"
	git push


test_exercises: start help ignorecase multiplefiles
generate_exercises: generate_grep generate_awk


Press space or the down key to continue

## Defining pipelines with Makefiles

Makefiles are a basic way to define pipelines of shell commands.

Nowadays there are more sophisticated tools available, but most of these are based on Makefiles.


A Makefile is a collection of "rules".

Each of these rules follows this basic syntax is:

```
target: prerequisites
    commands to execute
```

As you can see in the Makefile included, most of the rules allow to regenerate the exercise files, or to execute some commands without having to type them everytime.

For example, the rule "testrule" is associated to two echo commands.

## How to run Makefile rules

To execute a rule in the Makefile, simply type:

```
make [name of the rule]
```

For example:



In [25]:
make testrule

echo this is a Makefile rule
this is a Makefile rule
echo You can associate it to as many commands you want
You can associate it to as many commands you want


The program "make" will automatically detect any file named "Makefile" in the current directory, and execute any rule with the specific name.

Rules can also be nested together. For example the two rules "test_exercises" and "generate_exercises" at the beginning of the file are a way to call several other rules together.

## Dinner time!!
